**Text Scraping**

This notebook includes the code used to scrape the description pages for both Rhizome and MOMA websites. The resulting data is used for some visualisations as well as storytelling. For Rhizome extracting data was also deemed important to help create medium information for the artworks (based on certain keywords such as html, java, flash etc...)

In [1]:
#imports and path 
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
import pandas as pd
#imports for cleaning and keyword extraction
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
path = '/Users/laurentfintoni/Desktop/University/COURSE DOCS/YEAR 2/EPUB/PROJECT/EPDS/'

**Rhizome Scraping**
- After some trials with the html construction of the Rhizome artwork pages we decided to grab the div that includes all three possible descriptions (summary, artist statement, description) and clean the results 

In [7]:
#function to scrape summaries, descriptions and artists statements from Rhizome website 
def url_to_text_rhizome(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")
    accordion = [p.text.strip() for p in soup.find(id="AccordionDescriptionBody").find_all('div')]
    print(url)
    return accordion

In [ ]:
#grab URLs from artworks DF, scrape them and return them back to the DF
rhz_artworks_extra = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra.pkl')
urls = rhz_artworks_extra['URL'].to_list()
scrapes = [url_to_text_rhizome(u) for u in urls]
rhz_artworks_extra_text = rhz_artworks_extra.copy()
rhz_artworks_extra_text['Text'] = pd.Series(scrapes)
#fix an erroneous ID in original first round of scraping 
rhz_artworks_extra_text.loc[777, 'ID'] = '926, 1268'
rhz_artworks_extra_text = rhz_artworks_extra_text.astype(str)
rhz_artworks_extra_text.to_pickle(path+'Rhizome_data/rhizome_artworks_extra_text.pkl')

**MOMA Scraping**
- For MOMA URLs there was only one possible description on the page but the containing div is repeated elsewhereso we used its parent container to only extract what we needed 
- We show the process for one dept only, but this was repeated for all of them 

In [3]:
#load department DFs
moma_arch_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont.pkl')
moma_arch_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod.pkl')
moma_design_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_img_cont.pkl')
moma_design_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_img_mod.pkl')
moma_draw_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont.pkl')
moma_draw_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod.pkl')
moma_films_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont.pkl')
moma_films_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod.pkl')
moma_fluxus_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_cont.pkl')
moma_fluxus_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_mod.pkl')
moma_media_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont.pkl')
moma_media_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod.pkl')
moma_paint_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont.pkl')
moma_paint_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod.pkl')
moma_photo_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont.pkl')
moma_photo_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod.pkl')

In [4]:
#function to extract text if available w/ exclusions for 404 status, missing URLs, and pages w/ no despcription
def url_to_text_moma(url):
    if url != 'missing':
        page = requests.get(url)
        status = page.status_code
        if status != 404:
            soup = BeautifulSoup(page.content, "html.parser")
            try:
                if soup.find(class_="uneven-columns--work").find(class_='main-content') is not None:
                    text = soup.find(class_="uneven-columns--work").find(class_='main-content').find_all('p')
                else:
                    text = ''
            except AttributeError:
                text = ''
                pass
        else:
            text = '404'
    else:
        text = 'missing'     
    print(url)
    return text

In [47]:
#grap all links from a dept as a list 
links = moma_photo_mod['URL'].to_list()
#process links w/ function 
moma_photo_to_add = [url_to_text_moma(u) for u in links]
#add results back to a copy of the original DF
moma_photo_mod_text = moma_photo_mod.copy()
moma_photo_mod_text['Text'] = moma_photo_to_add
moma_photo_mod_text['Text'] = moma_photo_mod_text['Text'].astype(str)
moma_photo_mod_text.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')

**Rhizome Cleaning & Keywords**
- Clean the text of html parsing errors 
- Analyse most common words for custom stopwords
- Run text through nltk english stopwords and custom ones 
- Extract 20 keywords for each text and add them as a new column (to use for assessing mediums of artworks)

In [12]:
rhz_artworks_extra_text = pd.read_pickle(path+'Rhizome_data/rhizome_artworks_extra_text.pkl')


In [ ]:
#create a list of strings to remove from all scrapes
remove = ['description edit\\\\n\\\\n\\\\t\\\\t\\\\t\\\\t', '[', ']', '\\\\n\\\\n\\\\n\\\\', '\'description edit\', ', 'edit\\\\n\\\\n', '\'summary edit\', ', 'tttt', 'nn']
for char in remove:
    rhz_artworks_extra_text['Text'] = rhz_artworks_extra_text['Text'].str.replace(char, '')

#export cleaned version to pickle
rhz_artworks_extra_text_clean = rhz_artworks_extra_text.copy()
rhz_artworks_extra_text_clean.to_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean.pkl')

In [ ]:
#check top 50 keywords 
common = pd.Series(' '.join(rhz_artworks_extra_text_clean['Text']).split()).value_counts()[:50]
common

In [18]:
# create a list of stop words and add custom stopwords
stop_words = set(stopwords.words("english"))
new_words =  ['the', 'a ', 'rhizome', '\'attributed', 'summary', 'to:', '\'inception:', 'staff\',', '\'attribution:', '\'summary', 'staffinception:', '-', '2021\',', '\'legacy', '2001\',']
stop_words = stop_words.union(new_words)

In [19]:
#remove stopwords from cleanedtext 
rhz_artworks_extra_text_clean['Text'] = rhz_artworks_extra_text_clean['Text'].str.lower()
rhz_artworks_extra_text_clean['Text'] = rhz_artworks_extra_text_clean['Text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

In [ ]:
#check top 50 keywords again
common_clean = pd.Series(' '.join(rhz_artworks_extra_text_clean['Text']).split()).value_counts()[:50]
common_clean

In [23]:
#function to extract keywords and pass them to a new column
def get_keywords(row):
    some_text = row['Text']
    tokens = nltk.tokenize.word_tokenize(some_text)
    keywords = [keyword for keyword in tokens if keyword.isalpha() and not keyword in stop_words]
    keywords_string = ', '.join(keywords[0:20])
    return keywords_string

rhz_artworks_extra_text_clean['Keywords'] = rhz_artworks_extra_text_clean.apply(lambda row:get_keywords(row), axis=1) 

In [26]:
#export to new pickle for reuse
rhz_artworks_extra_text_clean.to_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean_stop_keywords.pkl')

In [ ]:
rhz_artworks_extra_text_clean.loc[rhz_artworks_extra_text_clean['Keywords'].str.contains('html', case=False)]

**MOMA Cleaning**
- Clean the text 
- Remove empty entries and duplicates from DFs

In [2]:
#load DFs with text
moma_arch_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text.pkl')
moma_arch_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only.pkl')
moma_draw_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text.pkl')
moma_draw_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text.pkl')
moma_films_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text.pkl')
moma_films_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text.pkl')
moma_fluxus_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_cont_text.pkl')
moma_fluxus_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_mod_text.pkl')
moma_media_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text.pkl')
moma_media_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text.pkl')
moma_paint_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text.pkl')
moma_paint_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text.pkl')
moma_photo_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text.pkl')
moma_photo_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')


In [ ]:
#DFs to list then clean them 
moma_texts = [moma_arch_cont_text, moma_draw_cont_text, moma_draw_mod_text, moma_films_cont_text, moma_films_mod_text, moma_fluxus_cont_text, moma_fluxus_mod_text, moma_media_cont_text, moma_media_mod_text, moma_paint_cont_text, moma_paint_mod_text, moma_photo_cont_text, moma_photo_mod_text]

remove = ['[', ']', '</p>', '<p>', '<strong>', '</strong>', '<em>', '</em>', '</br>']
for text in moma_texts:
    for char in remove:
        text['Text'] = text['Text'].str.replace(char, '')

#exception for DF that was formatted differently due to scraping error (text column only)
for char in remove:
    moma_arch_mod_text['text'] = moma_arch_mod_text['text'].str.replace(char, '')

In [4]:
#replace empty strings w/ missing value to filter useful results only
moma_arch_cont_text['Text'] = moma_arch_cont_text['Text'].replace([''], 'missing')
moma_arch_mod_text['text'] = moma_arch_mod_text['text'].replace([''], 'missing')
moma_draw_cont_text['Text'] = moma_draw_cont_text['Text'].replace([''], 'missing')
moma_draw_mod_text['Text'] = moma_draw_mod_text['Text'].replace([''], 'missing')
moma_films_cont_text['Text'] = moma_films_cont_text['Text'].replace([''], 'missing')
moma_films_mod_text['Text'] = moma_films_mod_text['Text'].replace([''], 'missing')
moma_fluxus_cont_text['Text'] = moma_fluxus_cont_text['Text'].replace([''], 'missing')
moma_fluxus_mod_text['Text'] = moma_fluxus_mod_text['Text'].replace([''], 'missing')
moma_media_cont_text['Text'] = moma_media_cont_text['Text'].replace([''], 'missing')
moma_media_mod_text['Text'] = moma_media_mod_text['Text'].replace([''], 'missing')
moma_paint_cont_text['Text'] = moma_paint_cont_text['Text'].replace([''], 'missing')
moma_paint_mod_text['Text'] = moma_paint_mod_text['Text'].replace([''], 'missing')
moma_photo_cont_text['Text'] = moma_photo_cont_text['Text'].replace([''], 'missing')
moma_photo_mod_text['Text'] = moma_photo_mod_text['Text'].replace([''], 'missing')

In [5]:
#save cleaned DFs back 
moma_arch_cont_text.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text.pkl')
moma_arch_mod_text.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only.pkl')
moma_draw_cont_text.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text.pkl')
moma_draw_mod_text.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text.pkl')
moma_films_cont_text.to_pickle(path+'MOMA_data/pickle/departments/films_cont_text.pkl')
moma_films_mod_text.to_pickle(path+'MOMA_data/pickle/departments/films_mod_text.pkl')
moma_fluxus_cont_text.to_pickle(path+'MOMA_data/pickle/departments/fluxus_cont_text.pkl')
moma_fluxus_mod_text.to_pickle(path+'MOMA_data/pickle/departments/fluxus_mod_text.pkl')
moma_media_cont_text.to_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text.pkl')
moma_media_mod_text.to_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text.pkl')
moma_paint_cont_text.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text.pkl')
moma_paint_mod_text.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text.pkl')
moma_photo_cont_text.to_pickle(path+'MOMA_data/pickle/departments/photo_cont_text.pkl')
moma_photo_mod_text.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')

In [13]:
#reduce DFs down to entries w/ only useful text and remove duplicates 
moma_arch_cont_text_final = moma_arch_cont_text.loc[moma_arch_cont_text['Text'] != 'missing']
moma_arch_cont_text_final = moma_arch_cont_text_final.drop_duplicates(subset=['Text'])
moma_arch_mod_text_final = moma_arch_mod_text.loc[moma_arch_mod_text['text'] != 'missing']
moma_arch_mod_text_final = moma_arch_mod_text_final.drop_duplicates(subset=['text'])
moma_draw_cont_text_final = moma_draw_cont_text.loc[moma_draw_cont_text['Text'] != 'missing']
moma_draw_cont_text_final = moma_draw_cont_text_final.drop_duplicates(subset=['Text'])
moma_draw_mod_text_final = moma_draw_mod_text.loc[moma_draw_mod_text['Text'] != 'missing']
moma_draw_mod_text_final = moma_draw_mod_text_final.drop_duplicates(subset=['Text'])
moma_films_cont_text_final = moma_films_cont_text.loc[moma_films_cont_text['Text'] != 'missing']
moma_films_cont_text_final = moma_films_cont_text_final.drop_duplicates(subset=['Text'])
moma_films_mod_text_final = moma_films_mod_text.loc[moma_films_mod_text['Text'] != 'missing']
moma_films_mod_text_final = moma_films_mod_text_final.drop_duplicates(subset=['Text'])
moma_fluxus_cont_text_final = moma_fluxus_cont_text.loc[moma_fluxus_cont_text['Text'] != 'missing']
moma_fluxus_cont_text_final = moma_fluxus_cont_text_final.drop_duplicates(subset=['Text'])
moma_fluxus_mod_text_final = moma_fluxus_mod_text.loc[moma_fluxus_mod_text['Text'] != 'missing']
moma_fluxus_mod_text_final = moma_fluxus_mod_text_final.drop_duplicates(subset=['Text'])
moma_media_cont_text_final = moma_media_cont_text.loc[moma_media_cont_text['Text'] != 'missing']
moma_media_cont_text_final = moma_media_cont_text_final.drop_duplicates(subset=['Text'])
moma_media_mod_text_final = moma_media_mod_text.loc[moma_media_mod_text['Text'] != 'missing']
moma_media_mod_text_final = moma_media_mod_text_final.drop_duplicates(subset=['Text'])
moma_paint_cont_text_final = moma_paint_cont_text.loc[moma_paint_cont_text['Text'] != 'missing']
moma_paint_cont_text_final = moma_paint_cont_text_final.drop_duplicates(subset=['Text'])
moma_paint_mod_text_final = moma_paint_mod_text.loc[moma_paint_mod_text['Text'] != 'missing']
moma_paint_mod_text_final = moma_paint_mod_text_final.drop_duplicates(subset=['Text'])
moma_photo_cont_text_final = moma_photo_cont_text.loc[moma_photo_cont_text['Text'] != 'missing']
moma_photo_cont_text_final = moma_photo_cont_text_final.drop_duplicates(subset=['Text'])
moma_photo_mod_text_final = moma_photo_mod_text.loc[moma_photo_mod_text['Text'] != 'missing']
moma_photo_mod_text_final = moma_photo_mod_text_final.drop_duplicates(subset=['Text'])

In [15]:
#pickle final results for visualisation 
moma_arch_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final.pkl')
moma_arch_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final.pkl')
moma_draw_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final.pkl')
moma_draw_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final.pkl')
moma_films_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final.pkl')
moma_films_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final.pkl')
moma_fluxus_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/fluxus_cont_text_final.pkl')
moma_fluxus_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/fluxus_mod_text_final.pkl')
moma_media_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final.pkl')
moma_media_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final.pkl')
moma_paint_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final.pkl')
moma_paint_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final.pkl')
moma_photo_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final.pkl')
moma_photo_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final.pkl')

In [18]:
#load stopwords and remove them
stop_words = set(stopwords.words("english"))
new_words =  ['.', 'one', 'two', 'also']
stop_words = stop_words.union(new_words)

moma_texts_final = [moma_arch_cont_text_final, moma_draw_cont_text_final, moma_draw_mod_text_final, moma_films_cont_text_final, moma_films_mod_text_final, moma_fluxus_cont_text_final, moma_fluxus_mod_text_final, moma_media_cont_text_final, moma_media_mod_text_final, moma_paint_cont_text_final, moma_paint_mod_text_final, moma_photo_cont_text_final, moma_photo_mod_text_final]

for text in moma_texts_final:
    text['Text'] = text['Text'].str.lower()
    text['Text'] = text['Text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

#exception 
moma_arch_mod_text_final['text'] = moma_arch_mod_text_final['text'].str.lower()
moma_arch_mod_text_final['text'] = moma_arch_mod_text_final['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

In [23]:
common_moma = pd.Series(' '.join(moma_arch_mod_text_final['text']).split()).value_counts()[:50]
common_moma

design           249
new              224
architecture     181
designed         166
modern           155
would            143
architectural    141
city             140
work             118
urban            108
first            100
house             98
form              92
building          92
used              91
could             91
art               88
poster            87
drawing           84
many              74
project           73
made              72
architect         71
like              65
series            64
exhibition        64
rather            62
forms             61
world             59
buildings         57
war               55
international     55
drawings          55
projects          54
create            54
elements          54
space             54
part              54
industrial        53
well              52
early             52
created           52
became            51
inspired          49
furniture         49
called            48
produced          48
plan         